In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy

In [3]:
from google.colab import drive
drive.mount('/content/drive')

BASE_ADDR = './drive/MyDrive/Colab Notebooks/'

Mounted at /content/drive


In [4]:
ratings = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ratings.csv")
movies = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/movies.csv")

In [5]:
movies = movies.rename(columns={"id":"movieId"})
movies["movieId"] = movies["movieId"].astype("int64")
print("Movies: ")
print(movies[['movieId', 'title']])

Movies: 
      movieId                                      title
0           1                           Toy Story (1995)
1           2                             Jumanji (1995)
2           3                    Grumpier Old Men (1995)
3           4                   Waiting to Exhale (1995)
4           5         Father of the Bride Part II (1995)
...       ...                                        ...
9737   193581  Black Butler: Book of the Atlantic (2017)
9738   193583               No Game No Life: Zero (2017)
9739   193585                               Flint (2017)
9740   193587        Bungo Stray Dogs: Dead Apple (2018)
9741   193609        Andrew Dice Clay: Dice Rules (1991)

[9742 rows x 2 columns]


In [6]:
ratings.pivot(index="userId", columns="movieId", values="rating")

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,2.5,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
def create_user_item_matrix(ratings) -> pd.DataFrame:
    mat = ratings.pivot(index="userId", columns="movieId", values="rating")
    # Treat movies rated by users as 'seen' and assign a value of 1
    mat[~mat.isna()] = 1
    # Treat movies not rated by users as 'unseen' and assign a value of 0
    mat.fillna(0, inplace=True)
    return mat

user_item_matrix = create_user_item_matrix(ratings)
print(user_item_matrix)

movieId  1       2       3       4       5       6       7       8       \
userId                                                                    
1           1.0     0.0     1.0     0.0     0.0     1.0     0.0     0.0   
2           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
3           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
4           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
5           1.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
...         ...     ...     ...     ...     ...     ...     ...     ...   
606         1.0     0.0     0.0     0.0     0.0     0.0     1.0     0.0   
607         1.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
608         1.0     1.0     1.0     0.0     0.0     0.0     0.0     0.0   
609         1.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
610         1.0     0.0     0.0     0.0     0.0     1.0     0.0     0.0   

movieId  9       10     

In [8]:
def get_svd_prediction(user_item_matrix, k):
    # Obtain U, sigma, V
    u, s, vh = scipy.sparse.linalg.svds(user_item_matrix.to_numpy(), k=k)
    # Reconstruct the original matrix using these components
    preds = np.dot(np.dot(u, np.diag(s)), vh)

    # Convert the result to a DataFrame and normalize values between 0 and 1
    preds = pd.DataFrame(preds, columns=user_item_matrix.columns, index=user_item_matrix.index)
    preds = (preds - preds.min()) / (preds.max() - preds.min())
    return preds

predictions = get_svd_prediction(user_item_matrix, k=64)
print(predictions)

movieId    1         2         3         4         5         6         7       \
userId                                                                          
1        0.428114  0.425604  0.348175  0.154023  0.243343  0.453558  0.219816   
2        0.265159  0.209954  0.215911  0.136243  0.255729  0.187805  0.250365   
3        0.228959  0.227407  0.233126  0.151581  0.254409  0.215054  0.248156   
4        0.410161  0.140274  0.153194  0.167781  0.308183  0.314373  0.374465   
5        0.407273  0.325954  0.205385  0.161983  0.298501  0.263657  0.275638   
...           ...       ...       ...       ...       ...       ...       ...   
606      0.690775  0.217291  0.250497  0.143019  0.212394  0.141855  0.905293   
607      0.559043  0.408386  0.269015  0.131456  0.193766  0.344019  0.271290   
608      0.549373  0.980488  0.807716  0.204844  0.222560  0.213664  0.331879   
609      0.383871  0.296912  0.196343  0.127700  0.240714  0.213925  0.246829   
610      0.628879  0.198641 

In [9]:
user_id = 602
user_movie_ids = ratings[ratings.userId == user_id].movieId
user_movies = movies[movies.movieId.isin(user_movie_ids)]
print(user_id, "유저가 본 영화 목록")
print(len(user_movies), user_movies)

# Retrieve user predictions and exclude already watched movies
user_predictions = predictions.loc[user_id].sort_values(ascending=False)
user_predictions = user_predictions[~user_predictions.index.isin(user_movie_ids)]
# Get the top 10 movies with the highest predicted values
user_predictions = user_predictions.head(10)
# Retrieve information about the top 10 recommended movies
user_recommendations = movies[movies.movieId.isin(user_predictions.index)]
user_recommendations["recommendation_score"] = user_predictions.values
print("Movies the user hasn't watched and recommended:")
print(user_recommendations)

602 유저가 본 영화 목록
135      movieId                                              title  \
1          2                                     Jumanji (1995)   
5          6                                        Heat (1995)   
9         10                                   GoldenEye (1995)   
10        11                     American President, The (1995)   
13        14                                       Nixon (1995)   
..       ...                                                ...   
606      761                                Phantom, The (1996)   
615      780               Independence Day (a.k.a. ID4) (1996)   
621      786                                      Eraser (1996)   
633      805                             Time to Kill, A (1996)   
660      861  Supercop (Police Story 3: Supercop) (Jing cha ...   

                               genres  
1          Adventure|Children|Fantasy  
5               Action|Crime|Thriller  
9           Action|Adventure|Thriller  
10             

<ipython-input-9-6d817d17b128>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_recommendations["recommendation_score"] = user_predictions.values


In [10]:
len(user_movies)

135

In [11]:
user_recommendations

,movieId,title,genres,recommendation_score
114,137,Man of the Year (1995),Documentary,0.870013
121,148,"Awfully Big Adventure, An (1995)",Drama,0.870013
150,178,Love & Human Remains (1993),Comedy|Drama,0.870013
202,236,French Kiss (1995),Action|Comedy|Romance,0.726822
421,484,Lassie (1994),Adventure|Children,0.716392
433,496,What Happened Was... (1994),Comedy|Drama|Romance|Thriller,0.701294
506,588,Aladdin (1992),Adventure|Animation|Children|Comedy|Musical,0.694326
529,618,Two Much (1995),Comedy|Romance,0.688483
539,636,Frisk (1995),Drama,0.686505
4224,6145,Venom (1982),Horror|Thriller,0.659162


In [12]:
class SVD:
    def __init__(self, ratings, movies, k):
        user_item_matrix = create_user_item_matrix(ratings)
        self.preds = get_svd_prediction(user_item_matrix, k)
        self.ratings = ratings
        self.movies = movies

    def get_recommendations(self, user_id, top_k=10):
        user_movie_ids = self.ratings[self.ratings.userId == user_id].movieId
        user_movies = self.movies[self.movies.movieId.isin(user_movie_ids)]

        # Retrieve user predictions and exclude already watched movies
        user_predictions = self.preds.loc[user_id].sort_values(ascending=False)
        user_predictions = user_predictions[~user_predictions.index.isin(user_movie_ids)]
        # Get information about the top_k recommended movies
        user_recommendations = self.movies[self.movies.movieId.isin(user_predictions.index)]
        user_recommendations["recommendation_score"] = user_predictions.values

        return user_recommendations if top_k is None else user_recommendations.head(top_k)

In [13]:
svd = SVD(ratings, movies, 64)

In [14]:
svd.get_recommendations(user_id)

<ipython-input-12-0081aceaa628>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_recommendations["recommendation_score"] = user_predictions.values


,movieId,title,genres,recommendation_score
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0.870013
2,3,Grumpier Old Men (1995),Comedy|Romance,0.870013
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,0.870013
4,5,Father of the Bride Part II (1995),Comedy,0.726822
6,7,Sabrina (1995),Comedy|Romance,0.716392
7,8,Tom and Huck (1995),Adventure|Children,0.701294
8,9,Sudden Death (1995),Action,0.694326
11,12,Dracula: Dead and Loving It (1995),Comedy|Horror,0.688483
12,13,Balto (1995),Adventure|Animation|Children,0.686505
14,15,Cutthroat Island (1995),Action|Adventure|Romance,0.659162


In [18]:
# Define a function to calculate F1 score
def f1_score(svd, user_id, target_movie_ids, k=10):

    # Get the top-k recommendations for the user
    # 주어진 사용자 ID에 대한 top-k 추천 목록을 생성
    rec = svd.get_recommendations(user_id, k)

    # Calculate the number of true positives
    # 추천 목록에 타겟 영화 세트에 포함된 영화가 있는지 확인
    tp = len(rec[rec.movieId.isin(target_movie_ids)])

    # Calculate the number of false positives
    fp = k - tp
    # Calculate the number of false negatives
    # 타겟 영화 세트에 포함된 영화 중 추천되지 않은 영화의 수를 계산
    fn = len(target_movie_ids) - tp
    # Calculate the precision

    precision = tp / (tp + fp)
    # Calculate the recall
    recall = tp / (tp + fn)

    # Calculate the F1 score
    f1 = 2 * precision * recall / (precision + recall)

    return f1

# Define a target movie set
target_movie_ids = [
    1, 2, 3, 4, 5, 6, 7, 8, 9, 10,
    101, 102, 103, 104, 105, 106, 107, 108, 109, 110,
]

# Calculate the F1 score for each user
for user_id in ratings.userId.unique():
    f1 = f1_score(svd, user_id, target_movie_ids)
    print(f"User {user_id}: {f1:.2f}")

# Calculate the average F1 score across all users
average_f1 = (
    sum([f1_score(svd, user_id, target_movie_ids) for user_id in ratings.userId.unique()])
    / len(ratings.userId.unique())
)
print(f"Average F1 score: {average_f1:.2f}")


<ipython-input-12-0081aceaa628>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_recommendations["recommendation_score"] = user_predictions.values
<ipython-input-12-0081aceaa628>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_recommendations["recommendation_score"] = user_predictions.values
<ipython-input-12-0081aceaa628>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

User 1: 0.47
User 2: 0.67
User 3: 0.67
User 4: 0.67
User 5: 0.60
User 6: 0.13
User 7: 0.60
User 8: 0.53
User 9: 0.67
User 10: 0.67
User 11: 0.53
User 12: 0.67
User 13: 0.67
User 14: 0.53
User 15: 0.60
User 16: 0.67
User 17: 0.60
User 18: 0.47
User 19: 0.33
User 20: 0.53
User 21: 0.47
User 22: 0.67
User 23: 0.60
User 24: 0.60
User 25: 0.67
User 26: 0.60
User 27: 0.53
User 28: 0.60
User 29: 0.67
User 30: 0.67
User 31: 0.40
User 32: 0.40
User 33: 0.53
User 34: 0.60
User 35: 0.67
User 36: 0.67
User 37: 0.67
User 38: 0.67
User 39: 0.67
User 40: 0.60
User 41: 0.67
User 42: 0.47


<ipython-input-12-0081aceaa628>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_recommendations["recommendation_score"] = user_predictions.values
<ipython-input-12-0081aceaa628>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_recommendations["recommendation_score"] = user_predictions.values
<ipython-input-12-0081aceaa628>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

User 43: 0.27
User 44: 0.47
User 45: 0.40
User 46: 0.53
User 47: 0.67
User 48: 0.67
User 49: 0.67
User 50: 0.60
User 51: 0.47
User 52: 0.67
User 53: 0.67
User 54: 0.60
User 55: 0.67
User 56: 0.60
User 57: 0.47
User 58: 0.47
User 59: 0.60
User 60: 0.67
User 61: 0.67
User 62: 0.53
User 63: 0.53
User 64: 0.47
User 65: 0.67
User 66: 0.53
User 67: 0.67
User 68: 0.20
User 69: 0.67
User 70: 0.67
User 71: 0.53
User 72: 0.67
User 73: 0.60
User 74: 0.67
User 75: 0.67
User 76: 0.60
User 77: 0.67
User 78: 0.60
User 79: 0.60
User 80: 0.67
User 81: 0.60
User 82: 0.40
User 83: 0.67
User 84: 0.33
User 85: 0.67


<ipython-input-12-0081aceaa628>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_recommendations["recommendation_score"] = user_predictions.values
<ipython-input-12-0081aceaa628>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_recommendations["recommendation_score"] = user_predictions.values
<ipython-input-12-0081aceaa628>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

...

<ipython-input-12-0081aceaa628>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_recommendations["recommendation_score"] = user_predictions.values
<ipython-input-12-0081aceaa628>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_recommendations["recommendation_score"] = user_predictions.values
<ipython-input-12-0081aceaa628>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

User 607: 0.60
User 608: 0.40
User 609: 0.53
User 610: 0.53


<ipython-input-12-0081aceaa628>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_recommendations["recommendation_score"] = user_predictions.values
<ipython-input-12-0081aceaa628>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_recommendations["recommendation_score"] = user_predictions.values
<ipython-input-12-0081aceaa628>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

Average F1 score: 0.59


<ipython-input-12-0081aceaa628>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_recommendations["recommendation_score"] = user_predictions.values
<ipython-input-12-0081aceaa628>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_recommendations["recommendation_score"] = user_predictions.values
<ipython-input-12-0081aceaa628>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th